In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 1) 데이터 준비와 전처리

In [3]:
import numpy as np
import os
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares


In [4]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [6]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies.to_csv("movie.csv", sep=',', na_rep='NaN')

# 2) 분석해봅시다

- ratings에 있는 유니크한 영화 갯수
- ratings에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개 (인기순)

In [9]:
ratings['movie_id'].nunique()

3628

In [10]:
ratings['user_id'].nunique()

6039

In [11]:
data = ratings.join(movies.set_index('movie_id'), on = 'movie_id')
data

,user_id,movie_id,rating,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...
1000203,6040,1090,3,956715518,Platoon (1986),Drama|War
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama


In [12]:
using_cols = ['user_id', 'movie_id', 'rating', 'title']
data = data[using_cols]

data

,user_id,movie_id,rating,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
...,...,...,...,...
1000203,6040,1090,3,Platoon (1986)
1000205,6040,1094,5,"Crying Game, The (1992)"
1000206,6040,562,5,Welcome to the Dollhouse (1995)
1000207,6040,1096,4,Sophie's Choice (1982)


In [13]:
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [14]:
# 평균적으로 몇 편의 영화를 봤는가
user_count = data.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

# 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해줍시다.

In [15]:
data['movie_id'].max()

3952

In [37]:
data[data['title'].isin(my_favorite)].drop_duplicates(['title'])

,user_id,movie_id,rating,title
33,1,588,4,Aladdin (1992)
34,1,1907,4,Mulan (1998)
40,1,1,5,Toy Story (1995)
58,2,648,4,Mission: Impossible (1996)
459,6,364,4,"Lion King, The (1994)"


In [38]:
my_favorite = ['Aladdin (1992)', 'Mulan (1998)', 'Toy Story (1995)', 'Mission: Impossible (1996)', 'Lion King, The (1994)']
my_movie_id = [588, 1907, 1, 648, 364]

my_movie = pd.DataFrame({'user_id':['Mymin']*5, 'movie_id':my_movie_id, 'title':my_favorite, 'rating':[5]*5})
if not data.isin({'user_id':['Mymin']})['user_id'].any():
    data = data.append(my_movie, ignore_index = True)
    
data.tail(10)

,user_id,movie_id,rating,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,Mymin,588,5,Aladdin (1992)
836479,Mymin,1907,5,Mulan (1998)
836480,Mymin,1,5,Toy Story (1995)
836481,Mymin,648,5,Mission: Impossible (1996)
836482,Mymin,364,5,"Lion King, The (1994)"


In [39]:
# my_favorite = ['You Are the Apple of My Eye (2012)', 'About Time (2013)', 'Secret (2007)', 'Avengers: Endgame (2019)', 'Captain America: Civil War (2016)']
# my_movie_id = [4000, 4001, 4002, 4003, 4004]

# my_movie = pd.DataFrame({'user_id':['Mymin']*5, 'movie_id':my_movie_id, 'title':my_favorite, 'rating':[5]*5})
# if not data.isin({'user_id':['Mymin']})['user_id'].any():
#     data = data.append(my_movie, ignore_index = True)
    
# data.tail(10)

# 4) CSR matrix를 직접 만들어봅시다.

In [40]:
# indexing

# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [49]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['Mymin'])
print(movie_to_idx['Aladdin (1992)'])

6039
33


In [42]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,rating,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,588,5,33
836479,6039,1907,5,34
836480,6039,1,5,40
836481,6039,648,5,58


In [43]:
# CSR matrix 생성
num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

# print(num_user)
# print(num_movie)

# csr_data = csr_matrix((data['count'], (data.user_id, data.title)))
csr_data = csr_matrix((data['rating'], (data.user_id, data.title)), shape = (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜봅시다.

In [44]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [45]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [46]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [47]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해보세요.

In [52]:
mymin, aladdin = user_to_idx['Mymin'], movie_to_idx['Aladdin (1992)']
mymin_vector, aladdin_vector = als_model.user_factors[mymin], als_model.item_factors[aladdin]

In [53]:
mymin_vector

array([ 0.08691279, -0.24727906, -0.21949898, -1.057681  ,  0.22694537,
        0.33375776, -0.25518382,  1.0842284 , -0.2519733 , -0.15705884,
       -0.48835355, -0.2632406 , -0.70724756,  0.92401415, -0.72871137,
       -0.24232124,  0.2689693 , -0.03985931,  0.41006917, -0.08750905,
        0.288638  , -0.34153807,  0.33293116, -0.4160397 , -0.41810215,
        0.36681393,  0.3668607 ,  0.6843855 ,  0.21717311,  1.0092217 ,
        0.58227986,  0.89028496, -0.0870057 , -0.34190005,  0.6299759 ,
       -0.34789756, -0.2174278 ,  0.76562375, -0.10813221, -0.85557604,
       -0.10971881, -0.53929204, -0.7703286 ,  0.11221115, -0.07362784,
       -0.93736106, -0.57905185, -0.1254428 , -0.7330215 ,  0.43687326,
       -0.4407267 , -0.6432106 ,  0.31545243, -0.8923135 , -0.5270272 ,
        0.20300964,  0.44728208, -0.10180058, -0.04778064, -0.18540205,
        0.41567692,  0.86460525, -0.11973619, -0.25381365, -0.10304177,
       -0.32052642, -0.669542  , -0.13668942,  0.5603733 ,  0.04

In [54]:
apple_vector

array([ 0.01547285,  0.00851217,  0.00505118, -0.02305987,  0.01301063,
        0.01307462,  0.01451155,  0.04342039, -0.00095889,  0.01414096,
       -0.00686042, -0.01572099, -0.00835942,  0.02203416, -0.02608767,
       -0.01543044,  0.02798953, -0.0048195 ,  0.01690296,  0.01019884,
        0.00708138,  0.00413811,  0.01310037,  0.00062477,  0.01462858,
        0.00053883,  0.02106412,  0.0448046 ,  0.03508935,  0.02977692,
        0.02334778,  0.01384089,  0.0124521 , -0.02030532,  0.01304448,
        0.00870289,  0.01427954,  0.02826467,  0.00101874, -0.02628857,
       -0.00174909, -0.0300508 , -0.00801376,  0.00430086, -0.00769432,
        0.01146896, -0.01365634, -0.0047795 , -0.00613326,  0.01356449,
        0.00202197, -0.00816734,  0.01874137, -0.01543513,  0.00682378,
        0.00905961,  0.01477204,  0.00587448,  0.000899  ,  0.00814764,
        0.00294438,  0.04216511,  0.01332689,  0.01082403,  0.00842327,
       -0.00348739, -0.02205533, -0.01450343,  0.02551997, -0.00

In [55]:
np.dot(mymin_vector, apple_vector)

0.69381416

In [56]:
# 선호목록에 없는 영화

heat = movie_to_idx['Heat (1995)']
heat_vector = als_model.item_factors[heat]

In [57]:
np.dot(mymin_vector, heat_vector)

-0.024548188

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천 받아봅시다.

In [58]:
favorite_movie = 'Aladdin (1992)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=10)
similar_movie

[(33, 1.0),
 (10, 0.8292283),
 (330, 0.7945016),
 (32, 0.58490825),
 (40, 0.58159626),
 (191, 0.5809435),
 (35, 0.5757306),
 (329, 0.5300039),
 (34, 0.5281114),
 (16, 0.5137353)]

In [59]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Aladdin (1992)',
 'Beauty and the Beast (1991)',
 'Lion King, The (1994)',
 'Hercules (1997)',
 'Toy Story (1995)',
 'Little Mermaid, The (1989)',
 'Hunchback of Notre Dame, The (1996)',
 'Anastasia (1997)',
 'Mulan (1998)',
 'Tarzan (1999)']

In [60]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [61]:
get_similar_movie('Mission: Impossible (1996)')

['Mission: Impossible (1996)',
 'True Lies (1994)',
 'Lost World: Jurassic Park, The (1997)',
 'Conspiracy Theory (1997)',
 'GoldenEye (1995)',
 'Con Air (1997)',
 'Mask of Zorro, The (1998)',
 'Rock, The (1996)',
 'Batman Returns (1992)',
 'Twister (1996)']

# 8) 내가 가장 좋아할만한 영화들을 추천 받아봅시다.

In [62]:
user = user_to_idx['Mymin']
# recommend에서는 user*item CSR Matrix를 받습니다.
model_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
model_recommended

[(10, 0.6090466),
 (50, 0.49650776),
 (4, 0.44435048),
 (16, 0.34808007),
 (191, 0.33072388),
 (30, 0.3161142),
 (32, 0.30279344),
 (322, 0.2724043),
 (35, 0.26921165),
 (284, 0.24971679),
 (8, 0.23836154),
 (474, 0.2316555),
 (1, 0.22924586),
 (548, 0.22919233),
 (851, 0.22817306),
 (841, 0.22274086),
 (67, 0.21504101),
 (329, 0.21288118),
 (25, 0.20935944),
 (138, 0.19955912)]

In [64]:
[idx_to_movie[i[0]] for i in model_recommended]

['Beauty and the Beast (1991)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Tarzan (1999)',
 'Little Mermaid, The (1989)',
 'Antz (1998)',
 'Hercules (1997)',
 'Babe (1995)',
 'Hunchback of Notre Dame, The (1996)',
 'Nightmare Before Christmas, The (1993)',
 'Snow White and the Seven Dwarfs (1937)',
 'Chicken Run (2000)',
 'James and the Giant Peach (1996)',
 'Fantasia (1940)',
 'Iron Giant, The (1999)',
 'Prince of Egypt, The (1998)',
 'Gladiator (2000)',
 'Anastasia (1997)',
 'Pocahontas (1995)',
 'True Lies (1994)']

In [65]:
# explain 메소드를 통해서 추천에 기여한 정도를 파악
glad = movie_to_idx['Gladiator (2000)']
explain = als_model.explain(user, csr_data, itemid=glad)

In [66]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Mission: Impossible (1996)', 0.16257308156074435),
 ('Lion King, The (1994)', 0.05109656930102327),
 ('Mulan (1998)', 0.019369967720954007),
 ('Aladdin (1992)', 0.007388559142958175),
 ('Toy Story (1995)', -0.02712110252665266)]

In [67]:
bnb = movie_to_idx['Beauty and the Beast (1991)']
explain = als_model.explain(user, csr_data, itemid=bnb)

In [68]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Lion King, The (1994)', 0.2205308839982098),
 ('Aladdin (1992)', 0.21356338503397576),
 ('Mulan (1998)', 0.09246450155267097),
 ('Toy Story (1995)', 0.06389369307107003),
 ('Mission: Impossible (1996)', 0.008662384174528111)]

# 회고

### - 이번 프로젝트에서 어려웠던 점.
    처음에 영화 데이터셋 내에 존재하지 않는 새로운 영화들로 추천을 받으려다 보니 vector의 값들이 너무 낮은 크기로 설정이 되었습니다. (스스로 Cold Start 상황을 만들어서 프로젝트를 진행하고 있었음..)
    
    생각해보니 기존 데이터셋에 없는 영화를 주면 판단기준이 없어서 추천 시스템이 제대로 동작할 리가 없는데, 이부분을 깊게 생각하지 못해서 쓸데없는 시간소요가 너무 많았습니다..
    
    
    
### - 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.
    쇼핑몰이나 넷플릭스 같은 데를 보면 이 상품을 검색했던 사람들이 찾아봤던 상품들, 이 드라마를 시청한 사람들이 많이 본 영화(드라마) 이런식으로 추천해주는 서비스가 오늘 수행한 노드에 기반해 짜여져 있다는 것을 직접 확인해보니 신기하기도 하고 한편으로는 엄청 어려운 기술이 아니라는 것도 알게 되었습니다. (물론 정밀한 추천 시스템을 만들기 위해서는 더 심화된 기술이 적용되어야 하겠지만 ㅎㅎ)



### - 루브릭 평가 지표를 맞추기 위해 시도한 것들.
    1) CSR matrix가 정상적으로 만들어졌다. (사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었는가?)
        
       네. user_id와 영화 title의 갯수를 기반으로 사이즈를 맞추어 CSR matrix를 생성하였습니다. count열의 type 문제로 인해 column을 불러오는 부분에서 에러가 생겨서 data.rating -> data['rating']으로 변경하여 진행하였습니다. shape 부분에서 나는 에러는 indexing을 통해서 정확한 갯수 파악과 연속적이지 않은 인덱싱을 맞춰주어 해결하였습니다.
    
    
    
    2) MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. (사용자와 아이템 벡터 내적수치가 의미있게 형성되었는가?)
        
        네. 모델 훈련에서 크게 에러가 난 부분은 없었으며, 훈련된 모델에서 10개의 영화를 추천받았습니다. dataset에 존재하는 영화들이 대부분 오래된 영화들이라 아는 영화들을 위주로 선호 리스트를 작성하다보니 주로 Animation 영화들이었는데, 이거에 기반해서 추천해준 영화들도 대부분이 Animation이어서 모델의 성능이 왠만큼 나오는구나 생각했습니다.     
        벡터값을 출력해보았을 때, -1에서 1사이의 값이 정상적으로 나왔고, 이를 바탕으로 내적해서 구한 값들 역시 연관성이 높은 영화의 경우 0.6938, 낮은 영화의 경우 -0.024 정도의 값을 받아볼 수 있었습니다.
    
    
    
    3) 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. (MF 모델이 예측한 유저 선호도 및 아이템간의 유사도, 기여도가 의미있게 측정되었다.)
        
        네. 입력해 준 선호 영화 리스트와 MF 모델이 예측한 저의 영화 선호도를 보았을 때, 장르의 유사성이 보였고, 이를 통해 기여도를 따로 출력해보았을 때, Gladiator 영화를 추천해준 것은 Mission: Impossible 영화의 기여도가 제일 높게 나왔고, Beauty and the Beast 영화를 추천해주었을 때는 Lion King의 기여도가 제일 높게 나온 것으로 보아 기여도 역시 의미있게 측정된 것 같습니다.
      
        